# MCNPy Tutorial: Input File

This tutorial explores the advanced functionality of MCNPy for working with MCNP input files. We'll cover how to:

- Parse and examine MCNP input files
- Work with materials and nuclides
- Convert between atomic and weight fractions
- Analyze perturbation cards
- Manipulate and create new materials

Let's start by importing the necessary modules and loading an example input file.

In [1]:
import mcnpy
import pandas as pd
import numpy as np
from pathlib import Path

# Setup paths
repo_root = Path.cwd().resolve().parent
data_dir = repo_root / 'examples' / 'data'

# Load example input files
input_file1 = data_dir / 'inputfile_example_1.i'
input_file2 = data_dir / 'inputfile_example_2.i'

## 1. Basic Input File Parsing

Let's start by parsing an MCNP input file and exploring the basic structure.

In [2]:
# Parse the first example input file
input_data = mcnpy.read_mcnp(input_file1)

# Display basic information about the input file
input_data

                      MCNP Input Data                       

------------------------------------------------------------
                         MATERIALS                          
------------------------------------------------------------
Number of materials:      4
Material IDs:             100000, 200100, 300000, 300001
Atomic fraction mats:     4

Use .materials to access material data.

------------------------------------------------------------
                       PERTURBATIONS                        
------------------------------------------------------------
Number of perturbations:  1760
Perturbation IDs:         1-1760
Reactions available:      1, 2, 4, 51, 52, 102, 103, 107
Methods available:        -3, -2, 1, 2, 3
Perturbed materials:      300001
Energy range:             1.00e-11 - 2.00e+01 MeV
Energy bins:              44
Energy structure:         SCALE44

Use .perturbation to access perturbation data.

### Input File Structure

The parsed input file is represented by an `Input` object containing:

- `materials`: A collection of material cards
- `perturbation`: A collection of perturbation cards

Let's explore each of these components in more detail.

## 2. Working with Materials

Materials in MCNP define the composition of different regions in your geometry. The `Materials` class in MCNPy provides tools for working with these definitions.

In [3]:
# Access the materials collection
materials = input_data.materials

# Display summary information about the materials collection
materials

                 MCNP Materials Collection                  

Total Materials: 4

------------------------------------------------------------
    ID    |   Nuclides    |     Type      |    Libraries     
------------------------------------------------------------
  100000  |       2       |    Atomic     |       06c        
  200100  |       4       |    Atomic     |       06c        
  300000  |      36       |    Atomic     |       06c        
  300001  |      36       |    Atomic     |       06c        
------------------------------------------------------------

Available methods:
- .to_weight_fractions() - Convert all materials to weight fractions
- .to_atomic_fractions() - Convert all materials to atomic fractions

Examples of accessing data:
- .mat[material_id] - Access a specific material

In [4]:
# Print materials in MCNP format
print(materials)

   MCNP Materials Collection (4 materials)

            MCNP Material (ID: 100000)            

Properties: nlib=06c
Total Nuclides: 2

--------------------------------------------------
   ZAID   | Element  |   Fraction    |  Libraries  
--------------------------------------------------
   7014   |    N     |  7.8848e-01   |      -      
   8016   |    O     |  2.1152e-01   |      -      
--------------------------------------------------

Available methods:
- .to_weight_fraction() - Convert material to weight fractions
- .to_atomic_fraction() - Convert material to atomic fractions
- .convert_natural_elements() - Convert natural elements into isotopic composition
- .copy(new_id) - Create a copy with a new material ID

Examples of accessing data:
- .nuclide[zaid] - Access a specific nuclide
- print(material) - Print material in MCNP format

            MCNP Material (ID: 200100)            

Properties: nlib=06c
Total Nuclides: 4

--------------------------------------------------
   

### Accessing Specific Materials

Individual materials can be accessed by their ID number:

In [5]:
# Get a list of all material IDs
material_ids = list(materials.mat.keys())
print(f"Available material IDs: {material_ids}")

# Access a specific material (replace ID with an actual ID from your file)
material_id = material_ids[0]  # Take the first material ID as an example
material = materials.mat[material_id]

# Display the material
print(f"\nMaterial {material_id} details:")
print(material)

Available material IDs: [100000, 200100, 300000, 300001]

Material 100000 details:
m100000 nlib=06c
	7014 7.884803e-01
	8016 2.115197e-01


### Accessing Nuclides in a Material

Each material contains nuclides with their atomic or weight fractions. You can access these nuclides and their properties directly:

In [6]:
# Print all nuclides in the material
print(f"Material {material_id} contains {len(material.nuclide)} nuclides:\n")

# Get a list of ZAIDs in the material
zaids = list(material.nuclide.keys())
print(f"ZAIDs in material: {zaids}\n")

# Access a specific nuclide (first one in the list)
zaid = zaids[0]
nuclide = material.nuclide[zaid]

# Display nuclide attributes
print(f"Details for nuclide ZAID {zaid}:")
print(f"  Element symbol: {nuclide.element}")
print(f"  Fraction: {nuclide.fraction:.6e}")
print(f"  Fraction type: {'Weight' if nuclide.fraction < 0 else 'Atomic'}")
print(f"  Neutron library: {nuclide.nlib if nuclide.nlib else 'Using material default'}")
print(f"  Photon library: {nuclide.plib if nuclide.plib else 'Using material default'}")

# Iterate through all nuclides and print basic information
print("\nAll nuclides in material:")
for zaid, nuclide in material.nuclide.items():
    fraction_type = "Weight" if nuclide.fraction < 0 else "Atomic"
    print(f"  ZAID {zaid} ({nuclide.element}): {abs(nuclide.fraction):.6e} ({fraction_type})")

Material 100000 contains 2 nuclides:

ZAIDs in material: [7014, 8016]

Details for nuclide ZAID 7014:
  Element symbol: N
  Fraction: 7.884803e-01
  Fraction type: Atomic
  Neutron library: Using material default
  Photon library: Using material default

All nuclides in material:
  ZAID 7014 (N): 7.884803e-01 (Atomic)
  ZAID 8016 (O): 2.115197e-01 (Atomic)


### Converting Between Atomic and Weight Fractions

MCNP uses positive values for atomic fractions and negative values for weight fractions. MCNPy allows you to convert between these formats:

In [7]:
# Create a copy of the material to avoid modifying the original
new_material = material.copy(new_id=material_id+1000)

# Determine current fraction type
current_type = "weight" if any(nuc.fraction < 0 for nuc in new_material.nuclide.values()) else "atomic"
print(f"Original material (with {current_type} fractions):")
print(new_material)

# Convert to the other fraction type
if current_type == "weight":
    new_material.to_atomic_fraction()
    print("\nMaterial converted to atomic fractions:")
else:
    new_material.to_weight_fraction()
    print("\nMaterial converted to weight fractions:")

# Display the converted material
print(new_material)

Original material (with atomic fractions):
m101000 nlib=06c
	7014 7.884803e-01
	8016 2.115197e-01

Material converted to weight fractions:
m101000 nlib=06c
	7014 -7.654500e-01
	8016 -2.345500e-01


### Converting Natural Elements to Isotopes

MCNPy can convert these natural elements (ZAIDs ending in '00') into their constituent isotopes based on natural abundances:

In [8]:
# First, let's create a new material with a natural element (carbon)
natural_mat = mcnpy.input.material.Mat(id=200, nlib="80c")

# Add natural carbon (ZAID 6000)
natural_mat.add_nuclide(zaid=6000, fraction=0.5)

# Add natural oxygen (ZAID 8000)
natural_mat.add_nuclide(zaid=8000, fraction=0.5)

print("Material with natural elements:")
print(natural_mat)

# Check which nuclides are natural elements
print("\nIdentifying natural elements:")
for zaid, nuclide in natural_mat.nuclide.items():
    print(f"ZAID {zaid} ({nuclide.element}): Is natural element? {nuclide.is_natural}")

# Create a copy of the material to show the conversion
expanded_mat = natural_mat.copy(new_id=201)

# Convert all natural elements to isotopes
expanded_mat.convert_natural_elements()

print("\nMaterial after converting natural elements to isotopes:")
print(expanded_mat)


Material with natural elements:
m200 nlib=80c
	6000 5.000000e-01
	8000 5.000000e-01

Identifying natural elements:
ZAID 6000 (C): Is natural element? True
ZAID 8000 (O): Is natural element? True

Material after converting natural elements to isotopes:
m201 nlib=80c
	6012 4.946500e-01
	6013 5.350000e-03
	8016 4.987850e-01
	8017 1.900000e-04
	8018 1.025000e-03


### Converting Specific Natural Elements

You can also convert just a specific natural element by specifying its ZAID:

In [9]:
# Create another copy of the original material with natural elements
partial_mat = natural_mat.copy(new_id=202)

# Convert only the carbon (ZAID 6000) to its isotopes
partial_mat.convert_natural_elements(zaid_to_expand=6000)

print("Material after converting only carbon to isotopes:")
print(partial_mat)

Material after converting only carbon to isotopes:
m202 nlib=80c
	8000 5.000000e-01
	6012 4.946500e-01
	6013 5.350000e-03


### Creating a New Material

You can create new materials from scratch using the MCNPy API:

In [10]:
from mcnpy.input.material import Mat, Materials

# Create a new material - water (H2O)
water = Mat(id=100, nlib="80c")

# Add hydrogen (atomic fraction 2/3)
water.add_nuclide(zaid=1001, fraction=2/3)

# Add oxygen (atomic fraction 1/3)
water.add_nuclide(zaid=8016, fraction=1/3)

# Create a materials collection with our new material
new_materials = Materials()
new_materials.add_material(water)

# Display the new material in MCNP format
print("Water with atomic fractions:")
print(water)

# Convert to weight fractions and display again
water.to_weight_fraction()
print("\nWater with weight fractions:")
print(water)

Water with atomic fractions:
m100 nlib=80c
	1001 6.666667e-01
	8016 3.333333e-01

Water with weight fractions:
m100 nlib=80c
	1001 -1.119149e-01
	8016 -8.880851e-01


### Batch Operations on Materials

The `Materials` container provides methods to perform operations on all materials at once:

In [11]:
# Create a copy of our materials collection
materials_copy = Materials()
for mat_id, mat in materials.mat.items():
    materials_copy.mat[mat_id] = mat.copy(new_id=mat_id)

# Convert all materials to atomic fractions
materials_copy.to_atomic_fractions()
print("\nMaterials after conversion to atomic fractions:")
print(materials_copy)

# Convert to weight fractions
materials_copy.to_weight_fractions()
print("\n\nMaterials after conversion to weight fractions:")
print(materials_copy)


Materials after conversion to atomic fractions:
   MCNP Materials Collection (4 materials)

            MCNP Material (ID: 100000)            

Properties: nlib=06c
Total Nuclides: 2

--------------------------------------------------
   ZAID   | Element  |   Fraction    |  Libraries  
--------------------------------------------------
   7014   |    N     |  7.8848e-01   |      -      
   8016   |    O     |  2.1152e-01   |      -      
--------------------------------------------------

Available methods:
- .to_weight_fraction() - Convert material to weight fractions
- .to_atomic_fraction() - Convert material to atomic fractions
- .convert_natural_elements() - Convert natural elements into isotopic composition
- .copy(new_id) - Create a copy with a new material ID

Examples of accessing data:
- .nuclide[zaid] - Access a specific nuclide
- print(material) - Print material in MCNP format

            MCNP Material (ID: 200100)            

Properties: nlib=06c
Total Nuclides: 4

-----

## 3. Working with Perturbations

MCNP perturbation cards allow you to calculate the effect of small changes to your model. MCNPy provides tools for analyzing these perturbations.

In [12]:
# Access the perturbation collection
perturbations = input_data.perturbation

# Display summary information
perturbations

                   MCNP Perturbation Data                   

Number of perturbations:  1760
Perturbation numbers:     1-1760
Particle types:           n
Reactions available:      1, 2, 4, 51, 52, 102, 103, 107
Methods available:        -3, -2, 1, 2, 3
Energy range:             1.00e-11 - 2.00e+01 MeV
Number of energy bins:    44
Energy structure:         SCALE44


Examples of accessing data:
- .pert[perturbation_number] - Access a specific perturbation

### Accessing Specific Perturbations

Individual perturbation cards can be accessed by their ID:

In [13]:
# Get a list of all perturbation IDs
pert_ids = list(perturbations.pert.keys())

if pert_ids:  # Check if we have any perturbations
    print(f"Available perturbation IDs: {pert_ids}")
    
    # Access a specific perturbation
    pert_id = pert_ids[0]  # Take the first perturbation ID as an example
    pert = perturbations.pert[pert_id]
    
    # Display the perturbation
    print(f"\nPerturbation {pert_id} details:")
    print(pert)
else:
    print("No perturbations found in this input file.")
    
    # If no perturbations in the current file, check the second example file
    print("\nLet's check if the second example file has perturbations...")
    input_data2 = mcnpy.read_mcnp(input_file2)
    
    if hasattr(input_data2, 'perturbation') and input_data2.perturbation:
        perturbations = input_data2.perturbation
        pert_ids = list(perturbations.pert.keys())
        
        if pert_ids:
            print(f"Found perturbations in the second file. IDs: {pert_ids}")
            pert_id = pert_ids[0]
            pert = perturbations.pert[pert_id]
            print(f"\nPerturbation {pert_id} details:")
            print(pert)

Available perturbation IDs: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216,

### Converting Perturbations to a DataFrame

For better visualization, perturbation data can be converted to a pandas DataFrame:

In [14]:
# Check if we have perturbations to analyze
if hasattr(input_data, 'perturbation') and input_data.perturbation and input_data.perturbation.pert:
    perturbations = input_data.perturbation
    
    # Convert to DataFrame
    pert_df = perturbations.to_dataframe()
    
    # Display the DataFrame
    print("Perturbation data in tabular format:")
    display(pert_df)
    
    # Extract unique reaction types
    reactions = perturbations.reactions
    print(f"\nUnique reaction types: {reactions}")
    
    # Extract energy bins
    energies = perturbations.pert_energies
    if energies:
        print(f"\nEnergy structure: {len(energies)-1} bins")
        print(f"Energy range: {min(energies):.3e} - {max(energies):.3e} MeV")
else:
    print("No perturbations available to analyze.")

Perturbation data in tabular format:


,particle,cell,material,rho,method,reaction,e_min,e_max
id,,,,,,,,
1,n,"3, 5, 7, 9",300001,0.160393,2,1,1.000000e-11,3.000000e-09
2,n,"3, 5, 7, 9",300001,0.160393,3,1,1.000000e-11,3.000000e-09
3,n,"3, 5, 7, 9",300001,0.160393,-2,1,1.000000e-11,3.000000e-09
4,n,"3, 5, 7, 9",300001,0.160393,-3,1,1.000000e-11,3.000000e-09
5,n,"3, 5, 7, 9",300001,0.160393,1,1,1.000000e-11,3.000000e-09
...,...,...,...,...,...,...,...,...
1756,n,"3, 5, 7, 9",300001,0.160393,2,107,8.187300e+00,2.000000e+01
1757,n,"3, 5, 7, 9",300001,0.160393,3,107,8.187300e+00,2.000000e+01
1758,n,"3, 5, 7, 9",300001,0.160393,-2,107,8.187300e+00,2.000000e+01



Unique reaction types: [1, 2, 4, 51, 52, 102, 103, 107]

Energy structure: 44 bins
Energy range: 1.000e-11 - 2.000e+01 MeV


## 4. Summary

In this tutorial, we've explored the MCNPy functionality for working with MCNP input files:

1. **Input Files**: Parsing and accessing the basic structure
2. **Materials**: 
   - Accessing material data and nuclides
   - Converting between atomic and weight fractions
   - Converting natural elements to isotopic composition
   - Creating new materials
   - Batch operations on multiple materials
3. **Perturbations**:
   - Accessing perturbation cards
   - Converting to tabular format with DataFrames
   - Analyzing energy structures

These tools enable detailed analysis and manipulation of MCNP input files, facilitating advanced simulation workflows and sensitivity studies.